In [ ]:
def analyze_gender_math(df, dataset_type, filename, all_results):
    """Аналізує різницю в оцінках з математики за статтю."""
    required_cols = ['math_score_100', 'gender']
    if not all(col in df.columns for col in required_cols):
        print(f"⚠️ Відсутні колонки {required_cols} у {filename}")
        return pd.Series(), pd.Series()
    
    df_math = df[df['math_score_100'].notna() & df['gender'].isin(['чоловіча', 'жіноча'])]
    male_scores = df_math[df_math['gender'] == 'чоловіча']['math_score_100']
    female_scores = df_math[df_math['gender'] == 'жіноча']['math_score_100']
    
    result = {'file': filename, 'dataset_type': dataset_type}
    if len(male_scores) >= 30 and len(female_scores) >= 30:
        t_stat, p_value = ttest_ind(male_scores, female_scores, equal_var=False, nan_policy='omit')
        result.update({
            'male_count': len(male_scores), 'female_count': len(female_scores),
            'male_avg': male_scores.mean(), 'female_avg': female_scores.mean(),
            't_stat': t_stat, 'p_value': p_value, 'significant': p_value < 0.05
        })
    else:
        result['error'] = f'Недостатньо даних: чоловіків={len(male_scores)}, жінок={len(female_scores)}'
    
    all_results.append(result)
    return male_scores, female_scores

In [ ]:
def analyze_abroad_results(df, dataset_type, filename, all_results):
    """Порівнює оцінки з математики за кордоном і в Україні (лише для НМТ)."""
    if dataset_type != 'NMT':
        return pd.Series(), pd.Series()
    
    required_cols = ['math_score_100', 'pt_region_name']
    if not all(col in df.columns for col in required_cols):
        print(f"⚠️ Відсутні колонки {required_cols} у {filename}")
        return pd.Series(), pd.Series()
    
    df_scores = df[df['math_score_100'].notna() & df['pt_region_name'].notna()]
    df_scores['location'] = df_scores['pt_region_name'].apply(
        lambda x: 'За кордоном' if isinstance(x, str) and any(c in x.lower() for c in ['польща', 'латвія']) else 'Україна'
    )
    
    abroad_scores = df_scores[df_scores['location'] == 'За кордоном']['math_score_100']
    ukraine_scores = df_scores[df_scores['location'] == 'Україна']['math_score_100']
    
    result = {'file': filename, 'dataset_type': dataset_type}
    if len(abroad_scores) >= 30 and len(ukraine_scores) >= 30:
        t_stat, p_value = ttest_ind(abroad_scores, ukraine_scores, equal_var=False, nan_policy='omit')
        result.update({
            'abroad_count': len(abroad_scores), 'ukraine_count': len(ukraine_scores),
            'abroad_avg': abroad_scores.mean(), 'ukraine_avg': ukraine_scores.mean(),
            't_stat': t_stat, 'p_value': p_value, 'significant': p_value < 0.05
        })
    else:
        result['error'] = f'Недостатньо даних: за кордоном={len(abroad_scores)}, Україна={len(ukraine_scores)}'
    
    all_results.append(result)
    return abroad_scores, ukraine_scores

In [ ]:
def analyze_subject_correlation(df, dataset_type, filename, all_results):
    """Аналізує кореляцію між усіма предметами."""
    subjects = [
        'ukrainian_score_100', 'math_score_100',
        'biology_score_100', 'chemistry_score_100', 'physics_score_100',
        'english_score_100', 'geography_score_100', 'ukrainian_literature_score_100',
        'history_score_100', 'french_score_100', 'german_score_100', 'spanish_score_100'
    ]
    available_subjects = [s for s in subjects if s in df.columns]
    if len(available_subjects) < 2:
        print(f"⚠️ Недостатньо предметів для кореляції у {filename}: {available_subjects}")
        return pd.DataFrame()
    
    df_subjects = df[available_subjects].dropna()
    
    result = {'file': filename, 'dataset_type': dataset_type}
    if len(df_subjects) >= 30:
        corr_matrix = df_subjects.corr(method='pearson').round(2)
        for i, s1 in enumerate(available_subjects):
            for s2 in available_subjects[i+1:]:
                result[f'{s1.replace("_score_100", "")}_{s2.replace("_score_100", "")}_corr'] = \
                    corr_matrix.loc[s1, s2]
        result['corr_matrix'] = corr_matrix  # Зберігаємо для графіка
    else:
        result['error'] = f'Недостатньо даних: {len(df_subjects)} записів'
    
    all_results.append(result)
    return df_subjects

In [ ]:
def analyze_additional_subjects(df, dataset_type, filename, all_results):
    """Визначає частоту вибору додаткових предметів."""
    subjects = [
        'biology_score_100', 'chemistry_score_100', 'physics_score_100',
        'english_score_100', 'geography_score_100', 'ukrainian_literature_score_100',
        'history_score_100', 'french_score_100', 'german_score_100', 'spanish_score_100',
    ]
    counts = {}
    for subject in subjects:
        if subject in df.columns:
            counts[subject.replace('_score_100', '')] = df[subject].notna().sum()
    
    result = {'file': filename, 'dataset_type': dataset_type, **counts}
    all_results.append(result)
    return counts

In [ ]:
def analyze_age_groups(df, dataset_type, filename, all_results):
    """Порівнює оцінки з математики за віком (17/18)."""
    required_cols = ['math_score_100', 'student_age']
    if not all(col in df.columns for col in required_cols):
        print(f"⚠️ Відсутні колонки {required_cols} у {filename}")
        return pd.Series(), pd.Series()
    
    df_age = df[df['math_score_100'].notna() & df['student_age'].isin([17, 18])]
    age_17_scores = df_age[df_age['student_age'] == 17]['math_score_100']
    age_18_scores = df_age[df_age['student_age'] == 18]['math_score_100']
    
    result = {'file': filename, 'dataset_type': dataset_type}
    if len(age_17_scores) >= 30 and len(age_18_scores) >= 30:
        t_stat, p_value = ttest_ind(age_17_scores, age_18_scores, equal_var=False, nan_policy='omit')
        result.update({
            'age_17_count': len(age_17_scores), 'age_18_count': len(age_18_scores),
            'age_17_avg': age_17_scores.mean(), 'age_18_avg': age_18_scores.mean(),
            't_stat': t_stat, 'p_value': p_value, 'significant': p_value < 0.05
        })
    else:
        result['error'] = f'Недостатньо даних: 17={len(age_17_scores)}, 18={len(age_18_scores)}'
    
    all_results.append(result)
    return age_17_scores, age_18_scores

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import chardet
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Налаштування папок
INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "results"
GRAPHS_FOLDER = os.path.join(RESULTS_FOLDER, "graphs")
for folder in [RESULTS_FOLDER, GRAPHS_FOLDER]:
    os.makedirs(folder, exist_ok=True)

# Налаштування стилю графіків
sns.set_style("whitegrid")

def detect_encoding(file_path):
    """Визначає кодування файлу."""
    try:
        with open(file_path, 'rb') as f:
            return chardet.detect(f.read(10000))['encoding'] or 'utf-8'
    except Exception as e:
        print(f"Помилка визначення кодування для {file_path}: {e}")
        return 'utf-8'

def detect_separator(file_path, encoding):
    """Визначає роздільник у CSV-файлі."""
    try:
        with open(file_path, encoding=encoding) as f:
            sample = f.read(2048)
            return csv.Sniffer().sniff(sample).delimiter
    except Exception as e:
        print(f"Помилка визначення роздільника для {file_path}: {e}")
        return ','

def load_csv(file_path):
    """Завантажує CSV-файл."""
    encoding = detect_encoding(file_path)
    sep = detect_separator(file_path, encoding)
    print(f"   Кодування: {encoding}, роздільник: '{sep}'")
    
    try:
        df = pd.read_csv(file_path, encoding=encoding, sep=sep, quotechar='"', on_bad_lines='skip')
        print(f"   Перші 5 колонок: {list(df.columns[:5])}")
        return df
    except Exception as e:
        print(f"Помилка читання {file_path}: {e}")
        return None

def detect_dataset_type(filename):
    """Визначає тип датасету за назвою файлу."""
    match = re.search(r'(\d{4})', filename)
    if match:
        year = int(match.group(1))
        return 'NMT' if year >= 2022 else 'ZNO'
    return None

# Збір даних
zno_dfs = []
nmt_dfs = []
all_gender_results = []
all_abroad_results = []
all_correlation_results = []
all_subjects_results = []
all_age_results = []

# Основний цикл
for filename in os.listdir(INPUT_FOLDER):
    if not filename.endswith(".csv"):
        continue
    
    path = os.path.join(INPUT_FOLDER, filename)
    print(f"🔄 Обробка {filename}...")
    
    try:
        df = load_csv(path)
        if df is None or df.empty:
            print(f"⚠️ Файл {filename} порожній або не вдалося завантажити")
            continue
        
        dataset_type = detect_dataset_type(filename)
        if not dataset_type:
            print(f"⚠️ Невідомий тип даних: {filename}")
            continue
        
        print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")
        
        # Аналіз питань
        gender_male, gender_female = analyze_gender_math(df, dataset_type, filename, all_gender_results)
        abroad_abroad, abroad_ukraine = analyze_abroad_results(df, dataset_type, filename, all_abroad_results)
        subjects_df = analyze_subject_correlation(df, dataset_type, filename, all_correlation_results)
        subjects_counts = analyze_additional_subjects(df, dataset_type, filename, all_subjects_results)
        age_17, age_18 = analyze_age_groups(df, dataset_type, filename, all_age_results)
        
        # Збір даних для об'єднаного аналізу
        if dataset_type == 'ZNO':
            zno_dfs.append((df, gender_male, gender_female, subjects_df, age_17, age_18))
        else:
            nmt_dfs.append((df, gender_male, gender_female, abroad_abroad, abroad_ukraine, subjects_df, subjects_counts, age_17, age_18))
    
    except Exception as e:
        print(f"❌ Помилка у файлі {filename}: {e}")
        import traceback
        traceback.print_exc()

# Збереження результатів по роках
pd.DataFrame(all_gender_results).to_csv(os.path.join(RESULTS_FOLDER, 'gender_math_analysis.csv'), index=False)
pd.DataFrame(all_abroad_results).to_csv(os.path.join(RESULTS_FOLDER, 'abroad_results_analysis.csv'), index=False)
pd.DataFrame(all_correlation_results).to_csv(os.path.join(RESULTS_FOLDER, 'subject_correlation_analysis.csv'), index=False)
pd.DataFrame(all_subjects_results).to_csv(os.path.join(RESULTS_FOLDER, 'additional_subjects_analysis.csv'), index=False)
pd.DataFrame(all_age_results).to_csv(os.path.join(RESULTS_FOLDER, 'age_groups_analysis.csv'), index=False)

# Об'єднаний аналіз
summary_gender = []
summary_abroad = []
summary_correlation = []
summary_subjects = []
summary_age = []

for dataset_type, dfs_data in [
    ('ZNO', zno_dfs),
    ('NMT', nmt_dfs)
]:
    if dfs_data:
        print(f"\n🔄 Об'єднуємо дані для {dataset_type}...")
        try:
            dfs = [d[0] for d in dfs_data]
            combined_df = pd.concat(dfs, ignore_index=True)
            
            # Питання 1: Гендер
            male_scores = pd.concat([d[1] for d in dfs_data if not d[1].empty], ignore_index=True)
            female_scores = pd.concat([d[2] for d in dfs_data if not d[2].empty], ignore_index=True)
            if len(male_scores) >= 30 and len(female_scores) >= 30:
                t_stat, p_value = ttest_ind(male_scores, female_scores, equal_var=False)
                summary_gender.append({
                    'dataset_type': dataset_type,
                    'male_count': len(male_scores), 'female_count': len(female_scores),
                    'male_avg': male_scores.mean(), 'female_avg': female_scores.mean(),
                    't_stat': t_stat, 'p_value': p_value, 'significant': p_value < 0.05
                })
            
            # Питання 2: За кордоном (лише НМТ)
            if dataset_type == 'NMT':
                abroad_scores = pd.concat([d[3] for d in dfs_data if not d[3].empty], ignore_index=True)
                ukraine_scores = pd.concat([d[4] for d in dfs_data if not d[4].empty], ignore_index=True)
                if len(abroad_scores) >= 30 and len(ukraine_scores) >= 30:
                    t_stat, p_value = ttest_ind(abroad_scores, ukraine_scores, equal_var=False)
                    summary_abroad.append({
                        'dataset_type': dataset_type,
                        'abroad_count': len(abroad_scores), 'ukraine_count': len(ukraine_scores),
                        'abroad_avg': abroad_scores.mean(), 'ukraine_avg': ukraine_scores.mean(),
                        't_stat': t_stat, 'p_value': p_value, 'significant': p_value < 0.05
                    })
            
            # Питання 3: Кореляція
            subjects_dfs = [d[3 if dataset_type == 'ZNO' else 5] for d in dfs_data if not d[3 if dataset_type == 'ZNO' else 5].empty]
            if subjects_dfs:
                combined_subjects = pd.concat(subjects_dfs, ignore_index=True)
                if len(combined_subjects) >= 30:
                    corr_matrix = combined_subjects.corr(method='pearson').round(2)
                    result = {'dataset_type': dataset_type, 'corr_matrix': corr_matrix}
                    available_subjects = combined_subjects.columns
                    for i, s1 in enumerate(available_subjects):
                        for s2 in available_subjects[i+1:]:
                            result[f'{s1.replace("_score_100", "")}_{s2.replace("_score_100", "")}_corr'] = \
                                corr_matrix.loc[s1, s2]
                    summary_correlation.append(result)
            
            # Питання 4: Додаткові предмети
            subjects_counts = [d[6 if dataset_type == 'NMT' else 3] for d in dfs_data]
            combined_counts = {}
            for counts in subjects_counts:
                for subject, count in counts.items():
                    if isinstance(count, (int, float)) and count > 0:
                        combined_counts[subject] = combined_counts.get(subject, 0) + count
            if combined_counts:
                summary_subjects.append({'dataset_type': dataset_type, **combined_counts})
            
            # Питання 5: Вікові групи
            age_17_scores = pd.concat([d[4 if dataset_type == 'ZNO' else 7] for d in dfs_data if not d[4 if dataset_type == 'ZNO' else 7].empty], ignore_index=True)
            age_18_scores = pd.concat([d[5 if dataset_type == 'ZNO' else 8] for d in dfs_data if not d[5 if dataset_type == 'ZNO' else 8].empty], ignore_index=True)
            if len(age_17_scores) >= 30 and len(age_18_scores) >= 30:
                t_stat, p_value = ttest_ind(age_17_scores, age_18_scores, equal_var=False)
                summary_age.append({
                    'dataset_type': dataset_type,
                    'age_17_count': len(age_17_scores), 'age_17_count': len(age_17_scores),
                    'age_17_avg': age_17_scores.mean(), 'age_18_avg': age_18_scores.mean(),
                    't_stat': t_stat, 'p_value': p_value, 'significant': p_value < 0.05
                })
        
        except Exception as e:
            print(f"❌ Помилка в об'єднаному аналізі {dataset_type}: {e}")

# Збереження об'єднаних результатів
pd.DataFrame(summary_gender).to_csv(os.path.join(RESULTS_FOLDER, 'gender_math_summary.csv'), index=False)
pd.DataFrame(summary_abroad).to_csv(os.path.join(RESULTS_FOLDER, 'abroad_results_summary.csv'), index=False)
pd.DataFrame(summary_correlation).to_csv(os.path.join(RESULTS_FOLDER, 'subject_correlation_summary.csv'), index=False)
pd.DataFrame(summary_subjects).to_csv(os.path.join(RESULTS_FOLDER, 'additional_subjects_summary.csv'), index=False)
pd.DataFrame(summary_age).to_csv(os.path.join(RESULTS_FOLDER, 'age_groups_summary.csv'), index=False)

# Вставте секцію генерації графіків тут

print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")

In [ ]:
# Генерація графіків
print("\n🔄 Генерація графіків...")
# Питання 1: Гендер (по роках)
for result in all_gender_results:
    if 'male_avg' in result and 'female_avg' in result:
        plt.figure(figsize=(8, 6))
        plt.bar(['Чоловіки', 'Жінки'], [result['male_avg'], result['female_avg']],
                color=['#36A2EB', '#FF6384'], edgecolor=['#2E8B57', '#C71585'])
        plt.title(f'Математика за статтю ({result["dataset_type"]}, {result["file"]})')
        plt.ylabel('Середня оцінка (0-200)')
        plt.savefig(os.path.join(GRAPHS_FOLDER, f'gender_math_{result["file"]}.png'), dpi=300, bbox_inches='tight')
        plt.close()

# Питання 1: Гендер (об'єднані)
for result in summary_gender:
    plt.figure(figsize=(8, 6))
    plt.bar(['Чоловіки', 'Жінки'], [result['male_avg'], result['female_avg']],
            color=['#36A2EB', '#FF6384'], edgecolor=['#2E8B57', '#C71585'])
    plt.title(f'Математика за статтю ({result["dataset_type"]}, Combined)')
    plt.ylabel('Середня оцінка (0-200)')
    plt.savefig(os.path.join(GRAPHS_FOLDER, f'gender_math_{result["dataset_type"]}_combined.png'), dpi=300, bbox_inches='tight')
    plt.close()

# Питання 2: За кордоном (по роках, лише НМТ)
for result in all_abroad_results:
    if 'abroad_avg' in result and 'ukraine_avg' in result:
        plt.figure(figsize=(8, 6))
        plt.bar(['За кордоном', 'Україна'], [result['abroad_avg'], result['ukraine_avg']],
                color=['#FFCE56', '#4BC0C0'], edgecolor=['#FFD700', '#20B2AA'])
        plt.title(f'Математика за місцем тестування (НМТ, {result["file"]})')
        plt.ylabel('Середня оцінка (0-200)')
        plt.savefig(os.path.join(GRAPHS_FOLDER, f'abroad_results_{result["file"]}.png'), dpi=300, bbox_inches='tight')
        plt.close()

# Питання 2: За кордоном (об'єднані, лише НМТ)
for result in summary_abroad:
    plt.figure(figsize=(8, 6))
    plt.bar(['За кордоном', 'Україна'], [result['abroad_avg'], result['ukraine_avg']],
            color=['#FFCE56', '#4BC0C0'], edgecolor=['#FFD700', '#20B2AA'])
    plt.title(f'Математика за місцем тестування (НМТ, Combined)')
    plt.ylabel('Середня оцінка (0-200)')
    plt.savefig(os.path.join(GRAPHS_FOLDER, f'abroad_results_NMT_combined.png'), dpi=300, bbox_inches='tight')
    plt.close()

# Питання 3: Кореляція (по роках)
for result in all_correlation_results:
    if 'corr_matrix' in result:
        corr_matrix = result['corr_matrix']
        subjects = [s.replace('_score_100', '') for s in corr_matrix.columns]
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                    xticklabels=subjects, yticklabels=subjects)
        plt.title(f'Кореляція предметів ({result["dataset_type"]}, {result["file"]})')
        plt.savefig(os.path.join(GRAPHS_FOLDER, f'subject_correlation_{result["file"]}.png'), dpi=300, bbox_inches='tight')
        plt.close()

# Питання 3: Кореляція (об'єднані)
for result in summary_correlation:
    if 'corr_matrix' in result:
        corr_matrix = result['corr_matrix']
        subjects = [s.replace('_score_100', '') for s in corr_matrix.columns]
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                    xticklabels=subjects, yticklabels=subjects)
        plt.title(f'Кореляція предметів ({result["dataset_type"]}, Combined)')
        plt.savefig(os.path.join(GRAPHS_FOLDER, f'subject_correlation_{result["dataset_type"]}_combined.png'), dpi=300, bbox_inches='tight')
        plt.close()

# Питання 4: Додаткові предмети (по роках)
for result in all_subjects_results:
    counts = {k: v for k, v in result.items() if k not in ['file', 'dataset_type'] and isinstance(v, (int, float)) and v > 0}
    if counts:
        plt.figure(figsize=(8, 6))
        plt.pie(list(counts.values()), labels=list(counts.keys()), colors=sns.color_palette('pastel'),
                autopct='%1.1f%%')
        plt.title(f'Популярність предметів ({result["dataset_type"]}, {result["file"]})')
        plt.savefig(os.path.join(GRAPHS_FOLDER, f'additional_subjects_{result["file"]}.png'), dpi=300, bbox_inches='tight')
        plt.close()
    else:
        print(f"⚠️ Пропущено графік для {result['file']}: немає коректних даних для предметів ({counts})")

# Питання 4: Додаткові предмети (об'єднані)
for result in summary_subjects:
    counts = {k: v for k, v in result.items() if k != 'dataset_type' and isinstance(v, (int, float)) and v > 0}
    if counts:
        plt.figure(figsize=(8, 6))
        plt.pie(list(counts.values()), labels=list(counts.keys()), colors=sns.color_palette('pastel'),
                autopct='%1.1f%%')
        plt.title(f'Популярність предметів ({result["dataset_type"]}, Combined)')
        plt.savefig(os.path.join(GRAPHS_FOLDER, f'additional_subjects_{result["dataset_type"]}_combined.png'), dpi=300, bbox_inches='tight')
        plt.close()
    else:
        print(f"⚠️ Пропущено графік для {result['dataset_type']}_combined: немає коректних даних для предметів ({counts})")

# Питання 5: Вікові групи (по роках)
for result in all_age_results:
    if 'age_17_avg' in result and 'age_18_avg' in result:
        plt.figure(figsize=(8, 6))
        plt.bar(['17 років', '18 років'], [result['age_17_avg'], result['age_18_avg']],
                color=['#FF6384', '#36A2EB'], edgecolor=['#DC143C', '#1E90FF'])
        plt.title(f'Математика за віком ({result["dataset_type"]}, {result["file"]})')
        plt.ylabel('Середня оцінка (0-200)')
        plt.savefig(os.path.join(GRAPHS_FOLDER, f'age_groups_{result["file"]}.png'), dpi=300, bbox_inches='tight')
        plt.close()

# Питання 5: Вікові групи (об'єднані)
for result in summary_age:
    plt.figure(figsize=(8, 6))
    plt.bar(['17 років', '18 років'], [result['age_17_avg'], result['age_18_avg']],
            color=['#FF6384', '#36A2EB'], edgecolor=['#DC143C', '#1E90FF'])
    plt.title(f'Математика за віком ({result["dataset_type"]}, Combined)')
    plt.ylabel('Середня оцінка (0-200)')
    plt.savefig(os.path.join(GRAPHS_FOLDER, f'age_groups_{result["dataset_type"]}_combined.png'), dpi=300, bbox_inches='tight')
    plt.close()

print(f"\n🎉 Генерація графіків завершена! Графіки збережено в '{GRAPHS_FOLDER}'")
